In [1]:
'''
This script shows how to predict stock prices using a basic RNN
'''
import tensorflow as tf
import numpy as np
import matplotlib
import os

tf.set_random_seed(777)  # reproducibility

def MinMaxScaler(data):
    numerator = data - np.min(data, 0)
    denominator = np.max(data, 0) - np.min(data, 0)
    # noise term prevents the zero division
    return numerator / (denominator + 1e-7)


# train Parameters
seq_length = 3
data_dim = 25
hidden_dim = 10
output_dim = 1
learning_rate = 0.001
iterations = 1000

# Open, High, Low, Volume, Close
xy = np.loadtxt('test2.csv', delimiter=',')
#xy = xy[::-1]  # reverse order (chronically ordered)
#xy = MinMaxScaler(xy)

x = xy
y = xy[:, [-1]]  

# build a dataset
dataX = []
dataY = []
# print(len(y))
cnt = 0

for i in range(0, len(y) , 3): # 0~360 1~361
    # 0 3 6 9 12 15 18 ....... 360
    _x = x[i:i + seq_length] # 3 4 7 12 19 28 
    print("__x",_x)
    _y = y[i + seq_length - 1]  # Present close price
    
    print(_x, "---------->", _y)
    dataX.append(_x)
    dataY.append(_y)
    cnt = cnt + 1
    print(cnt)

# train/test split
train_size = int(len(dataY) * 0.7)
test_size = len(dataY) - train_size
trainX, testX = np.array(dataX[0:train_size]), np.array(
    dataX[train_size:len(dataX)])
trainY, testY = np.array(dataY[0:train_size]), np.array(
    dataY[train_size:len(dataY)])

# input place holders
X = tf.placeholder(tf.float32, [None, seq_length, data_dim])
Y = tf.placeholder(tf.float32, [None, 1])

# build a LSTM network
cell = tf.contrib.rnn.BasicLSTMCell(
    num_units=hidden_dim, state_is_tuple=True, activation=tf.tanh)
outputs, _states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)
Y_pred = tf.contrib.layers.fully_connected(
    outputs[:, -1], output_dim, activation_fn=None)  # We use the last cell's output

# cost/loss
loss = tf.reduce_sum(tf.square(Y_pred - Y))  # sum of the squares

# optimizer
optimizer = tf.train.AdamOptimizer(learning_rate)
train = optimizer.minimize(loss)

# RMSE
targets = tf.placeholder(tf.float32, [None, 1])
predictions = tf.placeholder(tf.float32, [None, 1])
rmse = tf.sqrt(tf.reduce_mean(tf.square(targets - predictions)))

with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)

    # Training step
    for i in range(iterations):
        _, step_loss = sess.run([train, loss], feed_dict={
                                X: trainX, Y: trainY})
        print("[step: {}] loss: {}".format(i, step_loss))

    # Test step
    test_predict = sess.run(Y_pred, feed_dict={X: testX})
    # print(test_predict)
    rmse_val = sess.run(rmse, feed_dict={
                    targets: testY, predictions: test_predict})
    print("RMSE: {}".format(rmse_val)) # 표준 편차 (차이가 안날수록 좋음)

    # Plot predictions
   # plt.plot(testY)
   # plt.plot(test_predict)
   # plt.xlabel("Time Period")
   # plt.ylabel("Stock Price")
   # plt.show()

__x [[ 0.365418    0.968867    1.27799     0.3401047   1.071257    1.439453
   0.2868706   1.249042    1.524018    0.1041874   1.343816    1.488172
   0.120924    1.061671    1.418151   -0.01744073  1.105032    1.264825
  -0.1323207   1.068463    1.407233   -0.05474424  1.247415    1.50129     1.        ]
 [ 0.3569106   0.9342036   1.221614    0.3399327   1.069388    1.439209
   0.2896993   1.248845    1.526418    0.1115404   1.343782    1.491731
   0.1215006   1.060479    1.41998    -0.1454718   1.029472    1.216883
  -0.1220509   1.099057    1.459478   -0.04888039  1.248716    1.502196    1.        ]
 [ 0.3573198   0.9365862   1.226138    0.3400248   1.066854    1.438422
   0.2909402   1.248363    1.527336    0.1140588   1.342895    1.493951
   0.1222571   1.058832    1.421711   -0.0421894   1.067582    1.258059
  -0.1237714   1.063138    1.406741   -0.0469012   1.247383    1.503608    1.        ]]
[[ 0.365418    0.968867    1.27799     0.3401047   1.071257    1.439453
   0.2868706  

__x [[ 0.09008776  1.288773    1.194669    0.2788128   1.378017    1.422867
   0.194855    1.599553    1.499397    0.00851315  1.719358    1.471607
   0.01432335  1.275678    1.421922   -0.2543677   1.140713    1.41721
  -0.2394405   1.33672     1.48308    -0.1414232   1.580433    1.455712    1.        ]
 [ 0.0842275   1.429158    1.225664    0.2231893   1.411699    1.363346
   0.1830994   1.605734    1.478336    0.00557121  1.718177    1.47677
   0.00542325  1.274985    1.425209   -0.2599501   1.142045    1.426411
  -0.2415027   1.338447    1.489843   -0.1655993   1.59038     1.478206    1.        ]
 [ 0.2441124   1.278405    1.235677    0.3172582   1.400567    1.381909
   0.1907348   1.595058    1.484713    0.00395683  1.717256    1.475471
   0.00502726  1.273853    1.423146   -0.2575987   1.139211    1.41757
  -0.2406482   1.335939    1.483671   -0.1482361   1.580376    1.457989    1.        ]]
[[ 0.09008776  1.288773    1.194669    0.2788128   1.378017    1.422867
   0.194855    1.

__x [[ 0.1343955   1.340245    0.9164016   0.3078123   1.354295    1.065494
   0.2283709   1.567779    1.164303    0.04315715  1.707097    1.161854
   0.03282212  1.241509    1.132632   -0.217952    1.282112    1.000307
  -0.2284312   1.411499    1.179906   -0.1220127   1.562129    1.148808    1.        ]
 [ 0.08071704  1.271745    0.9512861   0.20973     1.393908    1.099328
   0.2190949   1.574476    1.173657    0.03260018  1.703771    1.161686
   0.03075904  1.239373    1.13055    -0.1784145   1.243878    0.9479059
  -0.2347782   1.368022    1.114625   -0.1310255   1.560341    1.153012    1.        ]
 [ 0.09258641  1.425958    0.9691919   0.3016909   1.380719    1.103437
   0.1898386   1.524086    1.114603    0.03208724  1.703339    1.160979
   0.02787882  1.227573    1.128899   -0.171601    1.221644    0.9172674
  -0.2289391   1.35717     1.106963   -0.1323899   1.560823    1.153334    1.        ]]
[[ 0.1343955   1.340245    0.9164016   0.3078123   1.354295    1.065494
   0.2283709

  -0.3358286   1.022204    1.325292   -0.2757363   1.198059    1.388276    1.        ]]
[[ 0.03926295  1.013725    1.184162    0.1411134   1.023196    1.322858
   0.07664502  1.211981    1.434056   -0.103382    1.203626    1.355622
  -0.09808116  0.9988535   1.336549   -0.2943572   0.9129767   1.212816
  -0.345474    1.028124    1.341666   -0.2868062   1.198115    1.397735    1.        ]
 [ 0.1022891   0.9581487   1.19111     0.17419     1.04925     1.382389
   0.07224348  1.205878    1.432857   -0.1037227   1.202654    1.356569
  -0.09741152  0.9972484   1.334455   -0.2814648   0.9069614   1.201575
  -0.3363373   1.023875    1.327975   -0.2785995   1.19839     1.391593    1.        ]
 [ 0.0221543   1.178895    1.12751     0.1313696   1.026525    1.27898
   0.09012388  1.206243    1.437617   -0.09773547  1.212276    1.350325
  -0.09459019  0.9955259   1.332737   -0.2806681   0.9052898   1.198593
  -0.3358286   1.022204    1.325292   -0.2757363   1.198059    1.388276    1.        ]] ---

[step: 0] loss: 40.23434066772461
[step: 1] loss: 27.62158203125
[step: 2] loss: 17.67703628540039
[step: 3] loss: 10.28874397277832
[step: 4] loss: 5.276268005371094
[step: 5] loss: 2.3791608810424805
[step: 6] loss: 1.2461103200912476
[step: 7] loss: 1.435278296470642
[step: 8] loss: 2.440676689147949
[step: 9] loss: 3.7567625045776367
[step: 10] loss: 4.964666843414307
[step: 11] loss: 5.7941813468933105
[step: 12] loss: 6.132870674133301
[step: 13] loss: 5.993934154510498
[step: 14] loss: 5.470282077789307
[step: 15] loss: 4.693136215209961
[step: 16] loss: 3.801246166229248
[step: 17] loss: 2.9201762676239014
[step: 18] loss: 2.1494996547698975
[step: 19] loss: 1.5559473037719727
[step: 20] loss: 1.1712614297866821
[step: 21] loss: 0.9940958619117737
[step: 22] loss: 0.9954609870910645
[step: 23] loss: 1.1269344091415405
[step: 24] loss: 1.3305104970932007
[step: 25] loss: 1.5485913753509521
[step: 26] loss: 1.7325615882873535
[step: 27] loss: 1.848669409751892
[step: 28] loss: 1.

[step: 306] loss: 0.10755490511655807
[step: 307] loss: 0.1068127304315567
[step: 308] loss: 0.10607866942882538
[step: 309] loss: 0.10535122454166412
[step: 310] loss: 0.10463163256645203
[step: 311] loss: 0.10391876846551895
[step: 312] loss: 0.10321350395679474
[step: 313] loss: 0.10251501947641373
[step: 314] loss: 0.10182344168424606
[step: 315] loss: 0.10113921761512756
[step: 316] loss: 0.10046198219060898
[step: 317] loss: 0.0997907817363739
[step: 318] loss: 0.09912684559822083
[step: 319] loss: 0.09846984595060349
[step: 320] loss: 0.0978190079331398
[step: 321] loss: 0.09717530012130737
[step: 322] loss: 0.09653718024492264
[step: 323] loss: 0.09590606391429901
[step: 324] loss: 0.09528157114982605
[step: 325] loss: 0.09466277807950974
[step: 326] loss: 0.09405029565095901
[step: 327] loss: 0.09344423562288284
[step: 328] loss: 0.0928443893790245
[step: 329] loss: 0.09225039184093475
[step: 330] loss: 0.09166252613067627
[step: 331] loss: 0.09108050167560577
[step: 332] loss

[step: 556] loss: 0.03380518779158592
[step: 557] loss: 0.03369761258363724
[step: 558] loss: 0.033590421080589294
[step: 559] loss: 0.03348381444811821
[step: 560] loss: 0.033378053456544876
[step: 561] loss: 0.033272676169872284
[step: 562] loss: 0.033167727291584015
[step: 563] loss: 0.03306379169225693
[step: 564] loss: 0.032960228621959686
[step: 565] loss: 0.032857272773981094
[step: 566] loss: 0.03275470435619354
[step: 567] loss: 0.03265279158949852
[step: 568] loss: 0.0325515940785408
[step: 569] loss: 0.03245069459080696
[step: 570] loss: 0.0323508083820343
[step: 571] loss: 0.03225123509764671
[step: 572] loss: 0.032151959836483
[step: 573] loss: 0.032053541392087936
[step: 574] loss: 0.03195557743310928
[step: 575] loss: 0.031858235597610474
[step: 576] loss: 0.03176146745681763
[step: 577] loss: 0.031665150076150894
[step: 578] loss: 0.03156939893960953
[step: 579] loss: 0.031474050134420395
[step: 580] loss: 0.03137950226664543
[step: 581] loss: 0.03128521144390106
[step:

[step: 804] loss: 0.01897411420941353
[step: 805] loss: 0.018942508846521378
[step: 806] loss: 0.018911099061369896
[step: 807] loss: 0.01887994073331356
[step: 808] loss: 0.018848635256290436
[step: 809] loss: 0.01881769299507141
[step: 810] loss: 0.018786581233143806
[step: 811] loss: 0.018755776807665825
[step: 812] loss: 0.018725087866187096
[step: 813] loss: 0.018694477155804634
[step: 814] loss: 0.01866394840180874
[step: 815] loss: 0.018633529543876648
[step: 816] loss: 0.018603121861815453
[step: 817] loss: 0.018572980538010597
[step: 818] loss: 0.018542900681495667
[step: 819] loss: 0.01851280778646469
[step: 820] loss: 0.018482981249690056
[step: 821] loss: 0.018453238531947136
[step: 822] loss: 0.01842339336872101
[step: 823] loss: 0.018393853679299355
[step: 824] loss: 0.01836436800658703
[step: 825] loss: 0.01833498105406761
[step: 826] loss: 0.01830575056374073
[step: 827] loss: 0.018276609480381012
[step: 828] loss: 0.018247369676828384
[step: 829] loss: 0.01821849681437